<a href="https://colab.research.google.com/github/yumzi41/ProjetL1/blob/master/exercices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [8]:
! git clone https://github.com/yumzi41/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

fatal: destination path 'exam_2025' already exists and is not an empty directory.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [4]:
mySeed = 200

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [5]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

**Exercice 1** *Une relation linéaire* \

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef `'inputs'` donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef `'targets'` renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle :  
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon $$  
où $\epsilon \sim \mathcal{N}(0,\eta)$. \

Nous allons estimer les paramètres $\theta_0$, $\theta_1$, $\theta_2$ et $\theta_3$ à l’aide de la méthode analytique des moindres carrés, et évaluer l'erreur quadratique moyenne (*Mean Squared Error*, MSE) sur les jeux d'entraînement et de test.


In [24]:
import numpy as np

# Génération des jeux de données
train_set, test_set = generate_dataset(mySeed=42)
X_train, t_train = train_set['inputs'], train_set['targets']
X_test, t_test = test_set['inputs'], test_set['targets']

# Ajout d'une colonne de 1 pour le biais θ0
X_train_aug = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_test_aug = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

# Estimation des paramètres : θ = (X^T X)^(-1) X^T t
theta = np.linalg.inv(X_train_aug.T @ X_train_aug) @ (X_train_aug.T @ t_train)

# Prédictions et calcul des erreurs quadratiques moyennes
t_train_pred = X_train_aug @ theta
t_test_pred = X_test_aug @ theta
mse_train = np.mean((t_train_pred - t_train) ** 2)
mse_test = np.mean((t_test_pred - t_test) ** 2)

print("Paramètres estimés (θ0, θ1, θ2, θ3) :", theta)
print(f"MSE sur le train : {mse_train:.3f}")
print(f"MSE sur le test  : {mse_test:.3f}")


Paramètres estimés (θ0, θ1, θ2, θ3) : [2.1316691  0.46735151 0.49115291 0.73864314]
MSE sur le train : 4.285
MSE sur le test  : 3.935


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

**Q2**  
Pour estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD, l’architecture appropriée dépend de la nature des données et de la relation entre les prédicteurs $(x, y, z)$ et la cible $t$.  

### Choix d'architecture  
- Une **couche linéaire simple** (sans couche cachée) est suffisante si la relation entre les prédicteurs et la cible est strictement linéaire :  
  \[
  t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon
  \]
  Cette architecture permet d’estimer directement les coefficients $\theta_k$ et offre une **interprétabilité totale**.  

- Une **architecture avec une couche cachée** (par exemple, $3 \rightarrow 16 \rightarrow 1$ avec activation ReLU) peut être utilisée si des **non-linéarités** sont présentes dans les données. Cela améliore la **flexibilité** et permet de capturer des relations plus complexes, mais au prix d’une **perte d’interprétabilité**.  

### Justification  
- **Expressivité** : Une couche cachée avec des activations non linéaires permet de modéliser des relations non linéaires. Cependant, pour une tâche de régression linéaire stricte, une seule couche linéaire suffit.  
- **Performances en généralisation** : Une architecture plus complexe risque de surapprendre, surtout avec un petit dataset. Une simple couche linéaire favorise une meilleure généralisation si la relation est purement linéaire.

### Conclusion  
Pour ce problème, où les données suivent une relation linéaire définie par $\theta_k$, une **architecture avec une seule couche linéaire** est idéale. Voici le code correspondant.


In [20]:

# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Réseau linéaire simple : 3 entrées (x, y, z), 1 sortie (t)
        self.fc = nn.Linear(3, 1)  # Pas de couche cachée, directement une sortie

    def forward(self, x):
        return self.fc(x)


**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [21]:
# On détecte si un GPU est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.001)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
        # Passage sur GPU
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        # Remise à zéro des gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = mySimpleNet(batch_inputs)

        # Calcul de la loss
        loss = criterion(outputs.squeeze(), batch_targets)

        # Backpropagation
        loss.backward()

        # Mise à jour des paramètres
        optimizer.step()

    # Affichage occasionnel de la loss
    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [50/500], Loss: 5.3792
Epoch [100/500], Loss: 4.4078
Epoch [150/500], Loss: 5.3685
Epoch [200/500], Loss: 3.8084
Epoch [250/500], Loss: 3.2484
Epoch [300/500], Loss: 3.7801
Epoch [350/500], Loss: 3.9642
Epoch [400/500], Loss: 4.7946
Epoch [450/500], Loss: 4.9167
Epoch [500/500], Loss: 3.9054


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

**Q4**  
Pour un **réseau purement linéaire** (une seule couche `nn.Linear(3, 1)`), les estimations des $\theta_k$ se trouvent dans :  
- Le **biais** de la couche linéaire pour $\theta_0$,  
- Les **poids** de la couche linéaire pour $\theta_1, \theta_2, \theta_3$.  

Dans le cas d’un réseau à **couche(s) cachée(s)**, il n’y a plus d’équivalence directe avec les $\theta_k$ d’une simple régression linéaire, car les poids sont combinés avec une **activation non linéaire**. Pour retrouver un modèle linéaire pur, on doit retirer la (les) couche(s) cachée(s).

Voici un exemple de code qui utilise une **seule couche linéaire** et montre l’extraction de $\theta_0, \theta_1, \theta_2, \theta_3$ :


In [26]:
# Extraction des poids (θ1, θ2, θ3) et du biais (θ0)
theta_nn = mySimpleNet.fc.weight.data.cpu().numpy().squeeze()  # shape = (3,)
theta0 = mySimpleNet.fc.bias.data.cpu().numpy().item()      # scalaire

print(f"θ0 (biais)   = {theta0:.4f}")
print(f"θ1 (poids x) = {theta_nn[0]:.4f}")
print(f"θ2 (poids y) = {theta_nn[1]:.4f}")
print(f"θ3 (poids z) = {theta_nn[2]:.4f}")



θ0 (biais)   = 2.0639
θ1 (poids x) = 0.4634
θ2 (poids y) = 0.4910
θ3 (poids z) = 0.8507


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [28]:
# Affichage des paramètres estimés avec la méthode des moindres carrés
print("Paramètres estimés avec la méthode des moindres carrés :")
print(f"θ0 : {theta[0]:.4f}")
print(f"θ1 : {theta[1]:.4f}")
print(f"θ2 : {theta[2]:.4f}")
print(f"θ3 : {theta[3]:.4f}")

print("\nParamètres estimés avec le réseau de neurones :")
print(f"θ0 : {theta0:.4f}")
print(f"θ1 : {theta_nn[0]:.4f}")
print(f"θ2 : {theta_nn[1]:.4f}")
print(f"θ3 : {theta_nn[2]:.4f}")


Paramètres estimés avec la méthode des moindres carrés :
θ0 : 2.1317
θ1 : 0.4674
θ2 : 0.4912
θ3 : 0.7386

Paramètres estimés avec le réseau de neurones :
θ0 : 2.0639
θ1 : 0.4634
θ2 : 0.4910
θ3 : 0.8507


### Commentaires :
- Les valeurs des \(\theta_k\) estimées par la méthode des **moindres carrés** sont très proches de celles obtenues avec le **réseau de neurones**.
- La légère différence entre les deux estimations est due au fait que la méthode des moindres carrés fournit une solution exacte pour un modèle linéaire, tandis que le réseau de neurones utilise une optimisation approchée via SGD.
- Cette comparaison montre que le réseau de neurones a bien appris la relation linéaire présente dans les données.


\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [30]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

In [31]:
# Nb de paramètres dans self.Down1: (calcul "à la main")

# Nb de paramètres au total:

import torch

# On réutilise la classe causalFCN telle que définie dans le code fourni
model = causalFCN()

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Nombre total de paramètres du réseau : {num_params}")


Nombre total de paramètres du réseau : 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.




**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?
**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.
**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.
**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)
**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant. , pour toutes ces questions  

\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?